# Computer Vision Course (2024 Spring) Homework 1

---

## Problem 5. Camera calibration.
Camera calibration is the process of determining the intrinsic and extrinsic parameters of a camera. One of the most commonly used methods is proposed by Zhang in 1999. In this problem, let's simply simulate the process of this algorithm.
We provide a copy of python code for the simulation of a perspective camera model and a 3D chessboard, which can be used to generate the corner points of chessboard images from different viewpoints. In this way, we can skip the process of corner detection and focus on the calibration processes.

1) *Intrinsic parameters calibration.* Please generate 2D-3D correspondences using the provided code. Then use them to estimate the intrinsic parameters of the pre-defined camera. Compare the results with the ground-truth values, and discuss how should we place the chessboard (or the camera) during the calibration to get better results. (15 points)
2) *Extrinsic parameters calibration.* Assume that we have captured two images with the **same** chessboard and camera from different viewpoints. However, the detected corner points are somehow noised. We use the 3D coordinates of chessboard as the world coordinates. Please design a method to estimate the extrinsic camera parameters of the two images. For qualitative and quantitative evaluation, please visualize your results by projecting the 3D chessboard corner points onto the two images with the estimated camera parameters, and calculate the reprojection error. (15 points)

- Data for extrinsic parameters calibration:
  - Files: `data/cv_hw_1_5_correspondences0.txt`, `data/cv_hw_1_5_correspondences1.txt`
  - Format: In each row, (x, y, X, Y, Z) are the 2D coordinates of chessboard corner points in image plane and the corressponding 3D coordinates in space.
  - Visualization examples: the 2D points are visualized in `data/chessboard0.jpg` and `data/chessboard1.jpg` correspondingly.

- Hint:
  - Your answer will be graded based on both the performance of the algorithm (in Code blocks) and the completeness of the discussion (in Markdown blocks or the PDF report).
  - You can use `cv2.calibrateCamera()` or `cv2.solvePnP` functions. At the same time, please explain the parameters in your report.
  - You are also encouraged to write your own code without using `cv2.findHomography()`, which may earn more points.

---

### A simple guide to use `CameraModel` and `Chessboard3D`

Required environment:
- `python 3.8` with `numpy`, `opencv-python` packages.

The `CameraModel` class is to simulate a perspective camera.
- `CameraModel.project(points3D)`: project 3D points onto the 2D image plane.
    - Parameters:
        - `points3D`: 3D coordinates of points in 3D space, type `np.ndarray`, size `(N,3)`.
    - Returns:
        - `points2D`: 2D coordinates of points in image plane, type `np.ndarray`, size `(N,2)`.

- `CameraModel.draw(points2D, out_img_path=None)`: draw 2D points and check if they are inside the image.
    - Parameters:
        - `points2D`: 2D coordinates of points in image plane, type `np.ndarray`, size `(N,2)`.
        - `out_img_path`: path to save the output image.
    - Returns:
        - if `True`: all 2D points are inside the image.
        - if `False`: some 2D points are outside the image.

- `CameraModel.move_<axis_name>_plus(step)`: move the camera along `axis_name` in world coordinates.
    - axis_name: `x_axis`, `y_axis`, `z_axis`
    - Parameters:
        - `step`: moving distance.

- `CameraModel.rotate_<angel_name>(degree)`: rotate the camera in camera coordinates.
    - camera coordinates: x = right, y = down, z = view direction. (right-hand system)
    - angel_name:
        - `pitch`: clockwise along the x-axis
        - `yaw`: clockwise along the y-axis
        - `roll`: clockwise along the z-axis
    - Parameters:
        - `degree`: rotation angle in degree, type `float`.

- `CameraModel.reset()`: reset the position and rotation of camera.

- `CameraModel.set_<K/R/T>(K/R/T)`: reset the parameters of camera with known values.


The `Chessboard3D` class is to simulate a simple 3D chessboard.
- `Chessboard3D.return_points()`: get the 3D coordinates of corner points.
    - Returns:
        - points3D: np.ndarray, (N,3)

In [82]:
# CameraModel

import numpy as np
import cv2

class CameraModel(object):
    """ a simple perspective camera with controllable intrinsic and extrinsic parameters.
    """
    dft_configs = {
        "fx": 200,
        "fy": 200,
        "cx": 256,
        "cy": 256,
        "pitch": 0, # degrees
        "yaw": 0, # degrees
        "roll": 0, # degrees
        "x": 0, # translation
        "y": 0,
        "z": 10,
    }
    def __init__(self, configs={}) -> None:
        self.configs = {**self.dft_configs, **configs}
        self.fx = self.configs["fx"]
        self.fy = self.configs["fy"]
        self.cx = self.configs["cx"]
        self.cy = self.configs["cy"]
        self.K = np.array([[self.fx, 0, self.cx], [0, self.fy, self.cy], [0, 0, 1]])

        self.pitch = self.configs["pitch"]
        self.yaw = self.configs["yaw"]
        self.roll = self.configs["roll"]
        self.R = self._get_rotation_matrix(self.pitch, self.yaw, self.roll)

        self.x = self.configs["x"]
        self.y = self.configs["y"]
        self.z = self.configs["z"]
        self.T = np.array([self.x, self.y, self.z])

    def show_K(self):
        print(f"K: {self.K}")
    
    def show_R(self):
        print(f"R: {self.R}")

    def show_T(self):
        print(f"T: {self.T}")

    def _get_rotation_matrix(self, pitch, yaw, roll):
        pitch = np.deg2rad(pitch)
        yaw = np.deg2rad(yaw)
        roll = np.deg2rad(roll)
        Rx = np.array([[1, 0, 0], [0, np.cos(pitch), -np.sin(pitch)], [0, np.sin(pitch), np.cos(pitch)]])
        Ry = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
        Rz = np.array([[np.cos(roll), -np.sin(roll), 0], [np.sin(roll), np.cos(roll), 0], [0, 0, 1]])
        
        return Rz @ Ry @ Rx

    def project(self, points3D):
        """ Project 3D points to 2D
        Args:
            points3D: 3D points, numpy array of shape (N, 3)
        """
        points3D = points3D.reshape(-1, 3)
        points3D = points3D.T
        points_ = self.R @ points3D + self.T.reshape(-1, 1)
        points_ = self.K @ points_
        points2D = points_ / points_[2, :]
        
        return points2D[:2, :].T # (N, 2)
    
    def draw(self, points, out_img_path=None, vbose=False):
        """ Draw 2D points on image
        Args:
            points: 2D points, numpy array of shape (N, 2)
        """
        img_W = 2*self.cx
        img_H = 2*self.cy
        img = np.zeros((img_H, img_W, 3), dtype=np.uint8)

        point_num = points.shape[0]
        outside_img_count = 0
        for point in points:
            x, y = point
            x, y = int(x), int(y)
            if 0 <= x < img_W and 0 <= y < img_H:
                img = cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
            else:
                outside_img_count += 1
                if vbose:
                    print(f"Point ({x}, {y}) is outside the image")
        if vbose:
            print(f"Outside image count: {outside_img_count}, while total points: {point_num}")

        if out_img_path:
            cv2.imwrite(out_img_path, img)
        
        if outside_img_count > 0: return False
        return True

    def move_x_axis_plus(self, step):
        self.x += step
        self.T = np.array([self.x, self.y, self.z])
    
    def move_y_axis_plus(self, step):
        self.y += step
        self.T = np.array([self.x, self.y, self.z])

    def move_z_axis_plus(self, step):
        self.z += step
        self.T = np.array([self.x, self.y, self.z])
    
    def rotate_pitch_plus(self, degree):
        self.pitch += degree
        self.R = self._get_rotation_matrix(self.pitch, self.yaw, self.roll)

    def rotate_yaw_plus(self, degree):
        self.yaw += degree
        self.R = self._get_rotation_matrix(self.pitch, self.yaw, self.roll)
    
    def rotate_roll_plus(self, degree):
        self.roll += degree
        self.R = self._get_rotation_matrix(self.pitch, self.yaw, self.roll)

    def reset(self):
        self.x = self.configs["x"]
        self.y = self.configs["y"]
        self.z = self.configs["z"]
        self.T = np.array([self.x, self.y, self.z])

        self.pitch = self.configs["pitch"]
        self.yaw = self.configs["yaw"]
        self.roll = self.configs["roll"]
        self.R = self._get_rotation_matrix(self.pitch, self.yaw, self.roll)
    
    def set_K(self, intrinsic_matrix):
        self.K = intrinsic_matrix
    
    def set_R(self, rotation_matrix):
        R = rotation_matrix
        
        # Extract pitch (rotation around x-axis)
        pitch = np.arctan2(R[2, 1], R[2, 2])
        # Extract yaw (rotation around y-axis)
        yaw = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
        # Extract roll (rotation around z-axis)
        roll = np.arctan2(R[1, 0], R[0, 0])

        # Convert radians to degrees
        self.pitch = np.degrees(pitch)
        self.yaw = np.degrees(yaw)
        self.roll = np.degrees(roll)
        self.R = R
    
    def set_T(self, translation):
        self.T = translation


In [83]:
# Chessboard3D

class Chessboard3D(object):
    """ a simple 3D chessboard, providing the coordinates of corner points.
    """
    dft_configs = {
        "square_size": 1,
        "rows": 8,
        "cols": 8,
        "z_plane": 0,
    }

    def __init__(self, configs={}) -> None:
        self.configs = {**self.dft_configs, **configs}
        self.square_size = self.configs["square_size"]
        self.rows = self.configs["rows"]
        self.cols = self.configs["cols"]
        self.z_plane = self.configs["z_plane"]
        self.points = self._generate_points()
    
    def _generate_points(self):
        points = []
        for row in range(self.rows):
            for col in range(self.cols):
                points.append([col*self.square_size, row*self.square_size, self.z_plane])
        return np.array(points) # (N, 3)

    def return_points(self):
        """ get corner points of the 3D chessboard
        Returns:
            points3D: np.ndarray, (N,3)
        """
        return self.points

### 5.1 Intrinsic parameters calibration

Steps:
1) Use `CameraModel` and `Chessboard3D` (with the default configuration) to generate 2D-3D correspondences.
2) Write code to estimate the camera intrinsic parameters.
3) Compare your results with the ground-truth values (in `CameraModel.dft_configs`).

---

#### Step 1: generate 2D-3D correspondences

In [84]:
# create class objects
camera = CameraModel()
chessboard = Chessboard3D()

# get the 3D coordinates in chessboard
coordinate_3D = chessboard.return_points()
# project 3D coordinates to 2D
projection_2D = camera.project(coordinate_3D)

#### Step 2&3: estimate the camera intrinsic parameters and compare results with the ground-truth values

* Here, we firstly use `cv2.calibrateCamera()` to estimate the camera intrinsic parameters. The `cv2.calibrateCamera` function in OpenCV is used to compute the camera calibration and distortion coefficients given a set of 3D real-world points and their corresponding 2D image points. The function prototype is:

`retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, imageSize, cameraMatrix, distCoeffs[, rvecs[, tvecs[, flags[, criteria]]]])`

* Parameter explanation:
    * `objectPoints`: A list of arrays of the 3D points in the real world. Each element of the list should be of the shape (N, 1, 3) or (1, N, 3) for N points in each scene.
    * `imagePoints`: A list of arrays of the corresponding 2D points in the image. Each element of the list should be of the shape (N, 1, 2) or (1, N, 2), matching the objectPoints.
    * `imageSize`: Size of the image used only to initialize the intrinsic camera matrix. It should be in the format of (width, height).
    * `distCoeffs`: Input/output vector of distortion coefficients.
* Return:
    * `cameraMatrix`: Input/output 2D array of the camera matrix
    $
    \left[
    \begin{matrix}
        f_x & 0 & c_x \\
        0 & f_y & c_y \\
        0 & 0 & 1
    \end{matrix}
    \right]
    $.
    * `distCoeffs`: Input/output vector of distortion coefficients.
    * `rvecs`: Output vector of rotation vectors (Rodrigues) estimated for each pattern view.
    * `tvecs`: Output vector of translation vectors estimated for each pattern view.


In [85]:
# The image size defined in CameraModel.draw() is (cy*2)*(cx*2)
image_size = (256*2, 256*2) 
# calibrateCamera() needs the input be float32, and shape (1, N, 3) and (1, N, 2)
ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera([coordinate_3D.astype('float32')], [projection_2D.astype('float32')], image_size, None, None)
print(cameraMatrix)
camera.show_K()

[[2.80066270e+19 0.00000000e+00 2.55527956e+02]
 [0.00000000e+00 2.80066270e+19 2.55551631e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
K: [[200   0 256]
 [  0 200 256]
 [  0   0   1]]


* We can see that in the return matrix focal length is too large, actually infinite. The reason is due to the 0 rotation.  During the calculation process, matrix elements such as $B_{11}$ is 0, then we induce the infinity focal length. It remains us to place the camera with some rotation in different directions.

![Formula of matrix B](./fig1.png)

* We give a rotation to camera, then

In [86]:
delta_rotation = 1
camera.rotate_pitch_plus(delta_rotation)
camera.rotate_yaw_plus(delta_rotation)
camera.rotate_roll_plus(delta_rotation)

# project 3D coordinates to 2D
projection_2D_1 = camera.project(coordinate_3D)
# calibrateCamera() needs the input be float32, and shape (1, N, 3) and (1, N, 2)
ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera([coordinate_3D.astype('float32')], [projection_2D_1.astype('float32')], image_size, None, None)
camera.show_K()
print("One homography with some rotation ")
print(cameraMatrix)

K: [[200   0 256]
 [  0 200 256]
 [  0   0   1]]
One homography with some rotation 
[[199.47669174   0.         255.50072632]
 [  0.         199.49409443 255.49925937]
 [  0.           0.           1.        ]]


* To get accurate result, we need more 2D-3D correspondances groups, i.e. more homography matrix. Let $n$ be the number of homography matrices. In the paper, we need to solve $b$ in the equation $Vb = 0$, where $V$ is a $2n\times 6$ matrix. If $n \geq 3$, we will have in general a unique solution $b$ defined up to a scale factor.
If $n = 2$, we can impose the skewless constraint $c = 0$, which is added as an additional equation.

In [87]:
delta_rotation = 3
camera.rotate_pitch_plus(delta_rotation)
camera.rotate_yaw_plus(delta_rotation)
camera.rotate_roll_plus(delta_rotation)

# project 3D coordinates to 2D
projection_2D_2 = camera.project(coordinate_3D)

delta_rotation = 10
camera.rotate_pitch_plus(delta_rotation)
camera.rotate_yaw_plus(delta_rotation)
camera.rotate_roll_plus(delta_rotation)

# project 3D coordinates to 2D
projection_2D_3 = camera.project(coordinate_3D)

camera.show_K()

ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera([coordinate_3D.astype('float32')]*2, [projection_2D_1.astype('float32'), projection_2D_2.astype('float32')], image_size, None, None)
print("2 homography: ")
print(cameraMatrix)

ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera([coordinate_3D.astype('float32')]*3, [projection_2D_1.astype('float32'), projection_2D_2.astype('float32'), projection_2D_3.astype('float32')], image_size, None, None)
print("3 homography: ")
print(cameraMatrix)

K: [[200   0 256]
 [  0 200 256]
 [  0   0   1]]
2 homography: 
[[200.00678335   0.         255.99933648]
 [  0.         200.00679381 256.0003592 ]
 [  0.           0.           1.        ]]
3 homography: 
[[199.99996534   0.         256.00008299]
 [  0.         199.99996223 255.99999008]
 [  0.           0.           1.        ]]


* Since the truth skewless constraint of $K$ is zero, so we can get a basically accurate result just using 2 homography matrices.

* Besides, we write own code to implement Zhang's calibration algorithm as the following. We can see that result has minor errors in skewless constraint and other parameters but basically correct.

In [88]:
def generate_v_ij(H_stack, i, j):
    """
    Generate intrinsic orthogonality constraints.
    """
    N = H_stack.shape[0]
    v_ij = np.zeros((N, 6))
    v_ij[:, 0] = H_stack[:, 0, i] * H_stack[:, 0, j]
    v_ij[:, 1] = H_stack[:, 0, i] * H_stack[:, 1, j] + H_stack[:, 1, i] * H_stack[:, 0, j]
    v_ij[:, 2] = H_stack[:, 1, i] * H_stack[:, 1, j]
    v_ij[:, 3] = H_stack[:, 2, i] * H_stack[:, 0, j] + H_stack[:, 0, i] * H_stack[:, 2, j]
    v_ij[:, 4] = H_stack[:, 2, i] * H_stack[:, 1, j] + H_stack[:, 1, i] * H_stack[:, 2, j]
    v_ij[:, 5] = H_stack[:, 2, i] * H_stack[:, 2, j]
    return v_ij

def svd_solve(A):
    """利用SVD求解齐次方程"""
    U, S, V_t = np.linalg.svd(A)
    # S已经从大到小排列了，方程解即奇异值最小的那列右奇异向量
    return V_t[np.argmin(S)]

def calc_homography(objectPoints, imagePoints):
    Hs = []
    for i in range(len(objectPoints)):
        A = []
        for j in range(len(objectPoints[i])):
            # convert to homogeneous coordinates
            x_i = np.append(objectPoints[i][j][:2], 1)
            # omega is 1
            A_i1 = np.hstack((np.zeros((3)), -1*x_i, imagePoints[i][j][1]*x_i))
            A_i2 = np.hstack((x_i, np.zeros((3)), -1*imagePoints[i][j][0]*x_i))
            A.append(A_i1)
            A.append(A_i2)
        A = np.array(A)
        Hs.append(svd_solve(A).reshape(3, 3))
    return Hs

def calc_intrinsic(objectPoints, imagePoints):
    Hs = calc_homography(objectPoints, imagePoints)
    N = len(Hs)
    H_stack = np.zeros((N, 3, 3))
    for idx, H in enumerate(Hs):
        H_stack[idx] = H
    # 注：实现时候我们-1
    v_00 = generate_v_ij(H_stack, i=0, j=0)
    v_01 = generate_v_ij(H_stack, i=0, j=1)
    v_11 = generate_v_ij(H_stack, i=1, j=1)

    V = np.zeros((2 * N, 6))
    V[:N] = v_01
    V[N:] = v_00 - v_11
    b = svd_solve(V)

    B11, B12, B22, B13, B23, B33 = b

    # calculate K
    v0 = (B12 * B13 - B11 * B23) / (B11 * B22 - B12 ** 2)
    lambda_ = B33 - (B13 ** 2 + v0 *(B12 * B13 - B11 * B23)) / B11
    alpha = np.sqrt(lambda_ / B11)
    beta = np.sqrt(lambda_ * B11 / (B11 * B22 - B12 **2))
    c = -1 * B12 * alpha ** 2 * beta / lambda_
    u0 = c * v0 / alpha - B13 * alpha ** 2 / lambda_

    K = np.array([[alpha, lambda_, u0],
                   [0.,    beta,  v0],
                   [0.,    0.,    1.]])
    return K

test_K = calc_intrinsic([coordinate_3D.astype('float32')]*3, [projection_2D_1.astype('float32'), projection_2D_2.astype('float32'), projection_2D_3.astype('float32')])
print(test_K)

[[2.00000480e+02 2.33819990e-01 2.55999904e+02]
 [0.00000000e+00 2.00000475e+02 2.56000124e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


### 5.2 Extrinsic parameters calibration

Steps:
1) Load 2D-3D correspondences from `data/cv_hw_1_5_correspondences0.txt` and `data/cv_hw_1_5_correspondences1.txt`.
2) Write code to calibrate the extrinsic camera parameters.
3) Visualize your results by projecting the chessboard corner points onto image planes.
4) Calculate the reprojection error and analysis the results.

---

#### Step 1: load 2D-3D correspondences

In [89]:
# Answer here.
data0 = np.loadtxt('data/cv_hw_1_5_correspondences0.txt')
data1 = np.loadtxt('data/cv_hw_1_5_correspondences1.txt')

#### Step 2: calibrate the extrinsic camera parameters

* In this step, we assume that we already know the instrinsic camera parameters, i.e. $K$ is known. We firstly use `cv2.solvePnP` function in OpenCV is utilized for estimating the pose of an object by finding the rotation and translation vectors that transform 3D points from the object coordinate frame to the camera coordinate frame. The function prototype is:

`retval, rvec, tvec = cv2.solvePnP(objectPoints, imagePoints, cameraMatrix, distCoeffs[, rvec[, tvec[, useExtrinsicGuess[, flags]]]])`

* Parameter explanation:
    * `objectPoints`: Array of object points in the object coordinate space. It is a floating-point array of shape (N, 1, 3), (1, N, 3), or (N, 3), where N is the number of points.
    * `imagePoints`: Array of corresponding image points. It is a floating-point array of shape (N, 1, 2), (1, N, 2), or (N, 2).
    * `cameraMatrix`: The camera's intrinsic parameters matrix
    $
    \left[
    \begin{matrix}
        f_x & 0 & c_x \\
        0 & f_y & c_y \\
        0 & 0 & 1
    \end{matrix}
    \right]
    $.
    * `distCoeffs`: Array of the camera's distortion coefficients. If the camera is undistorted or distortion has been corrected, you can pass None or an empty array.
* Output:
    * `rvec`: Output rotation vector that, together with tvec, transforms 3D points from the object coordinate system to the camera coordinate system. It can be initialized to provide an estimation to enhance speed and accuracy.
    * `tvec`: Output translation vector.

In [90]:
# already known the intrinsic matrix
K = np.array([[200, 0, 256], [0, 200, 256], [0, 0, 1]])

(success, rotation_vector, translation_vector) = cv2.solvePnP(np.array([coordinate_3D], dtype=np.double), np.array([projection_2D_1], dtype=np.double), K, distCoeffs = None,flags=cv2.SOLVEPNP_ITERATIVE)

# transfer the rotation_vector to rotation matrix
rotation_matrix = cv2.Rodrigues(rotation_vector)[0]

print("Rotation matrix:")
print("the truth value:")
print(camera._get_rotation_matrix(1,1,1))
print("the caculated value:")
print(rotation_matrix)

print("Translation vector:")
print("the truth value: ", camera.T)
print("the caculated value: ", translation_vector.reshape(3,))

Rotation matrix:
the truth value:
[[ 0.99969541 -0.01714521  0.01775168]
 [ 0.01744975  0.99970073 -0.01714521]
 [-0.01745241  0.01744975  0.99969541]]
the caculated value:
[[ 0.99969541 -0.01714521  0.01775168]
 [ 0.01744975  0.99970073 -0.01714521]
 [-0.01745241  0.01744975  0.99969541]]
Translation vector:
the truth value:  [ 0  0 10]
the caculated value:  [-6.26430683e-10 -4.17025181e-10  9.99999999e+00]


* We write own code to get the extrinsic parameters, it works well:

In [91]:
# already known the intrinsic matrix
K = np.array([[200, 0, 256], [0, 200, 256], [0, 0, 1]])

def calc_extrinsic(objectPoints, imagePoints, intrinsic_matrix):
    '''
    objectPoints: 1*N*3 array
    imagePoints: 1*N*2 array
    intrinsic_matrix: 3*3 matrix
    '''
    # calculate the homography
    homography_matrix = calc_homography(objectPoints, imagePoints)[0]
    h1 = homography_matrix[:, 0]
    h2 = homography_matrix[:, 1]
    h3 = homography_matrix[:, 2]
    lambda_ = 1 / np.linalg.norm(np.linalg.inv(intrinsic_matrix) @ h1.T)
    r1 = lambda_ * np.linalg.inv(intrinsic_matrix) @ h1.T
    r2 = lambda_ * np.linalg.inv(intrinsic_matrix) @ h2.T
    r3 = np.cross(r1.reshape(3,), r2.reshape(3,))
    t = lambda_ * np.linalg.inv(intrinsic_matrix) @ h3.T
    return np.vstack((r1, r2, r3)).T, t

R, T = calc_extrinsic([coordinate_3D], [projection_2D_1], K)

print("Rotation matrix:")
print("the truth value:")
print(camera._get_rotation_matrix(1,1,1))
print("the caculated value:")
print(R)

print("Translation vector:")
print("the truth value: ", camera.T)
print("the caculated value: ", T.reshape(3,))

Rotation matrix:
the truth value:
[[ 0.99969541 -0.01714521  0.01775168]
 [ 0.01744975  0.99970073 -0.01714521]
 [-0.01745241  0.01744975  0.99969541]]
the caculated value:
[[ 0.99969541 -0.01714521  0.01775168]
 [ 0.01744975  0.99970073 -0.01714521]
 [-0.01745241  0.01744975  0.99969541]]
Translation vector:
the truth value:  [ 0  0 10]
the caculated value:  [-6.92779167e-14 -3.01980663e-14  1.00000000e+01]


* To verify the correction of `cv2.solvePnP` and our code, we test the data generated from the ideal camera model. Now we calculate the extrinsic parameters for loaded 2D-3D correspondances.

In [92]:
R0, T0 = calc_extrinsic([data0[:, 2:]], [data0[:, :2]], K)
R1, T1 = calc_extrinsic([data1[:, 2:]], [data1[:, :2]], K)

print("Correspondances0:")
print("rotation matrix:")
print(R0)
print("translation vector: ", T0)

print("Correspondances1:")
print("rotation matrix:")
print(R1)
print("translation vector: ", T1)

Correspondances0:
rotation matrix:
[[ 1.00000000e+00 -2.48270526e-14  5.62436895e-13]
 [ 4.84003700e-13  1.00000000e+00  6.67797555e-14]
 [-5.62436895e-13 -6.67797555e-14  1.00000000e+00]]
translation vector:  [1.86517468e-13 2.89546165e-13 1.00000000e+01]
Correspondances1:
rotation matrix:
[[-9.64941363e-01  6.80376168e-02  2.53074663e-01]
 [ 3.63088875e-06 -9.64217060e-01  2.57589879e-01]
 [ 2.62465550e-01  2.48442400e-01  9.30412677e-01]]
translation vector:  [  4.99328035   4.99217421 -10.00280654]


#### Step 3: visualize results

In [93]:
camera0 = CameraModel()
camera0.R = R0
camera0.T = T0
# get the projection points using the estimated camera parameters and draw
camera0.draw(camera0.project(data0[:, 2:]), 'test_chessboard0.jpg')

camera1 = CameraModel()
camera1.R = R1
camera1.T = T1
# get the projection points using the estimated camera parameters and draw
camera1.draw(camera1.project(data1[:, 2:]), 'test_chessboard1.jpg')

True

#### Step 4: calculate the reprojection error and analysis the results

In [94]:
reprojection_error0 = np.sum((camera0.project(data0[:, 2:])-data0[:, :2])**2)
reprojection_error1 = np.sum((camera1.project(data1[:, 2:])-data1[:, :2])**2)

print('The reprojection error of correspondances0 is: ', reprojection_error0)
print('The reprojection error of correspondances1 is: ', reprojection_error1)

The reprojection error of correspondances0 is:  4.2300235179588005e-19
The reprojection error of correspondances1 is:  28.646919039094787


* We can see that the reprojection error of correspondances0 is 0 and correspondances1 is large. We can suppose that points of correspondances0 have accurate coordinates and points of correspondances1 have noisy coordinates.